In [5]:
import pandas as pd
pd.options.display.float_format = '{:.5f}'.format
import numpy as np
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

In [6]:
import os.path
def path_base(base_name):
    current_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
    print(current_dir)
    data_dir = current_dir.replace('notebook','data')
    print(data_dir)
    data_base = data_dir + '\\' + base_name
    print(data_base)
    return data_base

In [7]:
base = pd.read_csv(path_base('db_censo.csv'))

C:\MyPhyton\DataScience\notebook
C:\MyPhyton\DataScience\data
C:\MyPhyton\DataScience\data\db_censo.csv


In [8]:
base.shape

(32561, 15)

In [5]:
base.head(3)

,age,workclass,final-weight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loos,hour-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K


In [5]:
base.describe()

,age,final-weight,education-num,capital-gain,capital-loos,hour-per-week
count,32561.00000,32561.00000,32561.00000,32561.00000,32561.00000,32561.00000
mean,38.58165,189778.36651,10.08068,1077.64884,87.30383,40.43746
std,13.64043,105549.97770,2.57272,7385.29208,402.96022,12.34743
min,17.00000,12285.00000,1.00000,0.00000,0.00000,1.00000
25%,28.00000,117827.00000,9.00000,0.00000,0.00000,40.00000
50%,37.00000,178356.00000,10.00000,0.00000,0.00000,40.00000
75%,48.00000,237051.00000,12.00000,0.00000,0.00000,45.00000
max,90.00000,1484705.00000,16.00000,99999.00000,4356.00000,99.00000


# Previsores e classes

In [6]:
previsores = base.iloc[:,0:14].values


In [7]:
classe = base.iloc[:,14].values


# Transformação de categóricos para numéricos

In [8]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
labelencoder_previsores = LabelEncoder()

In [9]:
previsores[:, 1] = labelencoder_previsores.fit_transform(previsores[:, 1])
previsores[:, 3] = labelencoder_previsores.fit_transform(previsores[:, 3])
previsores[:, 5] = labelencoder_previsores.fit_transform(previsores[:, 5])
previsores[:, 6] = labelencoder_previsores.fit_transform(previsores[:, 6])
previsores[:, 7] = labelencoder_previsores.fit_transform(previsores[:, 7])
previsores[:, 8] = labelencoder_previsores.fit_transform(previsores[:, 8])
previsores[:, 9] = labelencoder_previsores.fit_transform(previsores[:, 9])
previsores[:, 13] = labelencoder_previsores.fit_transform(previsores[:, 13])

In [10]:
onehotencorder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [1,3,5,6,7,8,9,13])],remainder='passthrough')
previsores = onehotencorder.fit_transform(previsores).toarray()

In [11]:
previsores.shape

(32561, 108)

type(previsores)

# Deixar as variáveis na mesma escala
Importante em algoritimos que usam distancia Euclidiana como o Kmeans

In [12]:
labelencorder_classe = LabelEncoder()
classe = labelencorder_classe.fit_transform(classe)


In [13]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
previsores = scaler.fit_transform(previsores)

# Divisão em base de treinos e testes

In [14]:
from sklearn.model_selection import train_test_split
previsores_treinamento, previsores_teste, classe_treinamento, classe_teste = train_test_split(previsores, classe, test_size=0.15, random_state=0)

In [15]:
print(previsores_treinamento.shape)
print(classe_treinamento.shape)

(27676, 108)
(27676,)


In [16]:
print(previsores_teste.shape)
print(classe_teste.shape)

(4885, 108)
(4885,)


In [17]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [20]:
classificador = Sequential()
classificador.add(Dense(units=55,activation='relu',input_dim=108))
classificador.add(Dense(units=55,activation='relu'))
classificador.add(Dense(units=1,activation='sigmoid'))
classificador.compile(optimizer='adam',
                      loss = 'binary_crossentropy',
                     metrics=['accuracy'])

In [21]:
classificador.fit(previsores_treinamento,classe_treinamento,batch_size=10,epochs=100)
previsoes = classificador.predict(previsores_teste)
previsoes = (previsoes>0.5)

Epoch 1/100
27676/27676 [==============================] - 4s 133us/step - loss: 0.3443 - accuracy: 0.8410
Epoch 2/100
27676/27676 [==============================] - 4s 130us/step - loss: 0.3134 - accuracy: 0.8542
Epoch 3/100
27676/27676 [==============================] - 3s 108us/step - loss: 0.3053 - accuracy: 0.8575
Epoch 4/100
27676/27676 [==============================] - 3s 116us/step - loss: 0.2997 - accuracy: 0.8604
Epoch 5/100
27676/27676 [==============================] - 3s 113us/step - loss: 0.2953 - accuracy: 0.8629
Epoch 6/100
27676/27676 [==============================] - 3s 114us/step - loss: 0.2915 - accuracy: 0.8658
Epoch 7/100
27676/27676 [==============================] - 3s 121us/step - loss: 0.2879 - accuracy: 0.8677
Epoch 8/100
27676/27676 [==============================] - 3s 108us/step - loss: 0.2841 - accuracy: 0.8689
Epoch 9/100
27676/27676 [==============================] - 3s 113us/step - loss: 0.2801 - accuracy: 0.8684
Epoch 10/100
27676/27676 [===========

In [22]:
from sklearn.metrics import confusion_matrix,accuracy_score
precisao = accuracy_score(classe_teste,previsoes)
matriz = confusion_matrix(classe_teste,previsoes)

In [23]:
precisao

0.8143295803480041

In [24]:
matriz

array([[3272,  421],
       [ 486,  706]], dtype=int64)